In [ ]:
from agno.agent import Agent
from agno.team import Team
from agno.tools import tool
from agno.models.openai import OpenAIChat
from agno.knowledge.pdf import PDFKnowledgeBase
from agno.vectordb.qdrant import Qdrant
from agno.vectordb.search import SearchType
from dotenv import load_dotenv
import os
import requests
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
# === Tool-Based Agents ===
@tool(name="clickup_create", description="This tool is used to create a task on ClickUp", stop_after_tool_call=False)
def clickup_create(**kwargs):
    """
    This tool triggers a ClickUp task creation webhook and returns the webhook's response.

    Expected kwargs:
    - task_name (str) [REQUIRED]: Title of the ClickUp task.
    - content (str) [OPTIONAL]: Detailed description of the task.
    - assignee (str) [OPTIONAL]: User ID or email of the person to assign the task to.
    - priority (str/int) [OPTIONAL]: Task priority (e.g., "urgent", "high", "normal", or number code).
    - due_date (str) [OPTIONAL]: Due date in ISO 8601 format (e.g., "2025-07-31T18:00:00Z").

    The function sends these parameters to a pre-configured webhook and returns its response.
    """
    WEBHOOK_URL = "https://your-webhook-url.com/clickup/create"  # replace with your actual URL

    try:
        response = requests.post(WEBHOOK_URL, json=kwargs, timeout=10)
        response.raise_for_status()
        return response.json()  # or response.text if your webhook returns plain text
    except requests.RequestException as e:
        return {"error": str(e)}

clickup_agent = Agent(
    name="ClickUp Agent",
    model=OpenAIChat(id="gpt-4.1-nano"),
    tools=[clickup_create,clickup_update, clickup_get],
    show_tool_calls=True,
    read_chat_history=True,
    instructions=["Only respond to ClickUp-related tasks like creating, updating, or assigning tasks."]
)

@tool(name="slack_tool", stop_after_tool_call=False)
def slack_tool(**kwargs):
    return "✅ Slack tool triggered"

slack_agent = Agent(
    name="Slack Agent",
    model=OpenAIChat(id="gpt-4.1-nano"),
    tools=[slack_tool],
    show_tool_calls=True,
    read_chat_history=True,
    instructions=["Handle Slack messaging, tagging, and notifications."]
)

@tool(name="github_tool", stop_after_tool_call=False )
def github_tool(**kwargs):
    return "✅ GitHub tool triggered"

github_agent = Agent(
    name="GitHub Agent",
    model=OpenAIChat(id="gpt-4.1-nano"),
    tools=[github_tool],
    show_tool_calls=True,
    read_chat_history=True,
    instructions=["Respond to GitHub requests like creating issues, reviewing PRs, or commenting."]
)

@tool(name="devin_tool", stop_after_tool_call=False )
def devin_tool(**kwargs):
    return "✅ Devin tool triggered"

devin_agent = Agent(
    name="Devin Agent",
    model=OpenAIChat(id="gpt-4.1-nano"),
    tools=[devin_tool],
    show_tool_calls=True,
    read_chat_history=True,
    instructions=["Handle developer tasks like debugging, running tests, or building workflows."]
)

# === RAG Agent ===

collection_name = "new"
qdrant_url = "http://localhost:6333"

knowledge_base = PDFKnowledgeBase(
                path=f"data/user_abc",
                vector_db=Qdrant(
                    collection=collection_name,
                    url=qdrant_url,
                    # search_type=SearchType.hybrid,
                ),
            )

rag_agent = Agent(
    name="Knowledge Agent",
    model=OpenAIChat(id="gpt-4.1-nano"),
    knowledge=knowledge_base,
    search_knowledge=True,
    read_chat_history=True,
    instructions=[
            "You are a helpful assistant that can answer questions based on the provided PDF documents.",
            "Always cite the source when providing information from the documents.",
            "If you cannot find the answer in the documents, clearly state that.",
            "Be concise but thorough in your responses.",
            "Cite sources when possible. If nothing is found, say so clearly."
        ],
)
# === Orchestrator Team ===

orchestrator = Team(
    name="XO Orchestrator",
    model=OpenAIChat(id="gpt-4o"),
    members=[
        rag_agent,
        clickup_agent,
        slack_agent,
        github_agent,
        devin_agent
    ],
    instructions = [
                        "You are the coordinator for a team of specialized agents, each responsible for handling a specific type of request.",
                        
                        "You must determine which agent to delegate a user message to, based on the presence of explicit `@mentions`.",
                        
                        "Only trigger a specialized tool agent if the message includes one of the exact @mentions listed below. "
                        "If no such @mention is present, route the message to the `rag_agent` by default.",
                        
                        "Agent routing rules:",
                        "- Route to `clickup_agent` only if the message contains `@click-up`.",
                        "- Route to `slack_agent` only if the message contains `@slack`.",
                        "- Route to `github_agent` only if the message contains `@github`.",
                        "- Route to `copilot_agent` only if the message contains `@github-copilot`.",
                        "- Route to `devin_agent` only if the message contains `@devin`.",
                        "- Always route to `rag_agent` if the message contains `@XO`, or if no @mention is present at all.",
                        
                        "Do not infer tool usage from intent or keywords alone. If a message talks about GitHub or Slack but does not include the corresponding @mention, it should go to the `rag_agent`.",
                        
                        "If the message is ambiguous, vague, or lacking required context, delegate to the `rag_agent` and allow it to respond or request clarification.",
                        
                        "Your role is strictly to select the correct agent for execution. You do not generate or modify responses yourself."
                    ],
    mode="route",
    share_member_interactions=True,
    show_members_responses=True,
    read_team_history=True,
    enable_agentic_context=True,
    enable_team_history=True,
    num_of_interactions_from_history=5,
    show_tool_calls=True,
)
knowledge_base.load(recreate=False)

# === Example Usage ===

# if __name__ == "__main__":
#     response = orchestrator.run("Update the ClickUp task to fix the login bug and notify @krish on Slack.")
#     print(response)


INFO Embedder not provided, using OpenAIEmbedder as default.

INFO Creating collection

INFO Loading knowledge base

INFO Reading: Akreage Launch Pad Whitepaper

INFO Added 6 documents to knowledge base

INFO Reading: Akreage_Launchpad_Scope_of_Work (1)

INFO Added 2 documents to knowledge base

INFO Reading: Plan of Action

INFO Added 5 documents to knowledge base

In [4]:
def chat_with_orchestrator(agent):
    print("🧠 XO Orchestrator is ready. Type 'exit' to quit.\n")
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() in ["exit", "quit"]:
            print("👋 Goodbye!")
            break
        try:
            agent.print_response(user_input,stream=True)
            # response = agent.run(user_input)
            # print(f"🧠 XO: {response.content}\n")
        except Exception as e:
            print(f"❌ Error: {e}\n")
if __name__ == "__main__":
    chat_with_orchestrator(orchestrator)

🧠 XO Orchestrator is ready. Type 'exit' to quit.



Output()

Output()

Output()

👋 Goodbye!


In [98]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.storage.sqlite import SqliteStorage
from agno.models.openai import OpenAIChat
from agno.knowledge.pdf import PDFKnowledgeBase
from agno.vectordb.qdrant import Qdrant
from dotenv import load_dotenv
import os
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
user_id="user_finance2"
collection_name = "user_finance"
qdrant_url = "http://localhost:6333"
sql_db=SqliteStorage(table_name="agent_sessions", db_file="tmp/data.db")
knowledge_base = PDFKnowledgeBase(
                path=f"data/user_finance",
                vector_db=Qdrant(
                    collection=collection_name,
                    url=qdrant_url,
                    # search_type=SearchType.hybrid,
                ),
            )

rag_agent = Agent(
    name="Knowledge Agent",
    model=OpenAIChat(id="gpt-4.1-nano"),
    knowledge=knowledge_base,
    storage=storage,
    session_id=user_id,
    search_knowledge=True,
    read_chat_history=True,
    add_history_to_messages=True,
    # instructions=[
    #         "You are a helpful assistant that can answer questions based on the provided PDF documents.",
    #         "Always cite the source when providing information from the documents.",
    #         "If you cannot find the answer in the documents, clearly state that.",
    #         "Be concise but thorough in your responses.",
    #         "Cite sources when possible. If nothing is found, say so clearly."
    #     ],
    instructions = [
                    "You are a helpful assistant that answers questions using only the content from the provided PDF documents.",
                    "Do not guess or hallucinate. If the answer is not explicitly or implicitly found in the documents, respond with 'The answer is not available in the provided documents.'",
                    "Always include an in-text citation indicating the source document and relevant section (e.g., page number or heading) when referencing information.",
                    "Be concise, accurate, and informative. Avoid unnecessary elaboration not supported by the documents.",
                    "Use clear and professional language appropriate for academic or business use.",
                    "Do not include external knowledge or personal opinions. Only rely on the content available in the documents.",
                    "Structure multi-point answers in bullet or numbered format where applicable for better readability."
                ],
)

clickup_agent = Agent(
    name="ClickUp Agent",
    model=OpenAIChat(id="gpt-4.1-nano"),
    show_tool_calls=True,
    storage=storage,
    session_id=user_id,
    read_chat_history=True,
    add_history_to_messages=True,
    instructions=["create a format of clickup task based on the emory and user query, if its create task, the output a json containing task title, description, and due date",
                  "if the task is to update then output the fields to edit",
                  "if the task is to get task then output the title"]
)

slack_agent = Agent(
    name="Slack Agent",
    model=OpenAIChat(id="gpt-4.1-nano"),
    show_tool_calls=True,
    storage=storage,
    session_id=user_id,
    read_chat_history=True,
    add_history_to_messages=True,
    instructions=["Handle Slack messaging, tagging, and notifications."]
)
agent=Agent(
    name="Agent",
    model=OpenAIChat(id="gpt-4.1-nano"),
    show_tool_calls=True,
    storage=storage,
    session_id=user_id,
    read_chat_history=True,
    add_history_to_messages=True,
    instructions=["Handle general conversational queries"]
)
knowledge_base.load(recreate=False)

INFO Embedder not provided, using OpenAIEmbedder as default.

INFO Loading knowledge base

INFO Reading: leac203

INFO Added 0 documents to knowledge base

In [ ]:

rag_agent.print_response("",stream=True)

Output()

In [93]:
clickup_agent.print_response("what are all the questions/input i have asked till now",stream=True)

Output()

In [97]:
slack_agent.print_response("what do i do at XO",stream=True)

Output()

In [ ]:
agent.print_response("",stream=True)

Output()

In [109]:
from agno.agent import Agent
from agno.memory.v2.db.sqlite import SqliteMemoryDb
from agno.memory.v2.memory import Memory
from agno.models.openai import OpenAIChat
from agno.storage.sqlite import SqliteStorage
from rich.pretty import pprint
# UserId for the memories
user_id = "ava"
# Database file for memory and storage
db_file = "tmp/agent.db"

# Initialize memory.v2
memory = Memory(
    # Use any model for creating memories
    model=OpenAIChat(id="gpt-4.1"),
    db=SqliteMemoryDb(table_name="user_memories", db_file=db_file),
)
# Initialize storage
storage = SqliteStorage(table_name="agent_sessions", db_file=db_file)

# Initialize Agent
memory_agent = Agent(
    model=OpenAIChat(id="gpt-4.1"),
    # Store memories in a database
    memory=memory,
    # Give the Agent the ability to update memories
    enable_agentic_memory=True,
    # OR - Run the MemoryManager after each response
    enable_user_memories=True,
    # Store the chat history in the database
    storage=storage,
    # Add the chat history to the messages
    add_history_to_messages=True,
    # Number of history runs
    num_history_runs=3,
    markdown=True,
)

memory.clear()
memory_agent.print_response(
    "My name is Ava and I like to ski.",
    user_id=user_id,
    stream=True,
    stream_intermediate_steps=True,
)
print("Memories about Ava:")
pprint(memory.get_user_memories(user_id=user_id))

memory_agent.print_response(
    "I live in san francisco, where should i move within a 4 hour drive?",
    user_id=user_id,
    stream=True,
    stream_intermediate_steps=True,
)
print("Memories about Ava:")
pprint(memory.get_user_memories(user_id=user_id))

ImportError: cannot import name 'ToolExecution' from 'agno.models.response' (c:\documents\GEN AI\Agno\venv\Lib\site-packages\agno\models\response.py)

In [36]:
agent.print_response("help me ideate a draft for a github issue to fix the background issue, create title and its content",stream=True)

Output()

In [ ]:
agent1 = Agent(storage=storage,session_id="User1",read_chat_history=True,)
agent1.print_response("What is my name",stream=True)

Output()

In [38]:
agent1.print_response("Mention details you know about me",stream=True)

Output()

In [41]:
agent2 = Agent(storage=storage,session_id="User1",read_chat_history=True,)

In [42]:
agent2.print_response("What is my name",stream=True)

Output()